In [1]:
import tensorflow as tf

import tensorflow.keras as keras
import tensorflow.keras.utils as utils
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import ta 
import datetime

In [21]:

csvFileAddress = 'csv/FTSE100_H1_201607071400_202206242200.csv'

DJ = pd.read_csv(csvFileAddress,delim_whitespace=True)

DJ['<ISGREEN>'] =  DJ['<CLOSE>'] > DJ['<OPEN>']
DJ['<SIZE>'] =  (DJ['<CLOSE>'] - DJ['<OPEN>']) / DJ['<CLOSE>'] *100
DJ['<VOLATILITY>'] =  DJ['<HIGH>'] - DJ['<LOW>'] /DJ['<CLOSE>']

DJ.drop(['<VOL>'],axis=1,inplace=True)

DJ.fillna(0,inplace=True)

DJ['<EMA30>']= ta.trend.ema_indicator( DJ['<CLOSE>'],window=30)
DJ['<EMA50>']= ta.trend.ema_indicator( DJ['<CLOSE>'],window=50)
DJ['<EMA200>']= ta.trend.ema_indicator( DJ['<CLOSE>'],window=200)

def Upper(e):
    if e['<ISGREEN>'] : 
        return e['<HIGH>']-e['<CLOSE>']
    return e['<HIGH>']-e['<OPEN>']

def Lower(e):
    if e['<ISGREEN>'] : 
        return e['<OPEN>']-e['<LOW>']
    return e['<CLOSE>']-e['<LOW>']


DJ['<UPPER>'] = DJ.apply(Upper,axis=1)
DJ['<LOWER>'] = DJ.apply(Lower,axis=1)


In [22]:

# Adding TA indicators : 

from ta.volatility import BollingerBands,KeltnerChannel,average_true_range
from ta.trend import MACD

DJ['<RSI>'] = ta.momentum.rsi(DJ['<CLOSE>'],window=15,fillna=1) / 100
# DJ['<CCI>'] = ta.trend.cci(close=DJ['<CLOSE>'],high=DJ['<HIGH>'],low=DJ['<LOW>'],window=14,fillna=1)


keltner = KeltnerChannel(close=DJ['<CLOSE>'],high=DJ['<HIGH>'],low=DJ['<LOW>'],window=50,window_atr=20,multiplier=3,fillna=1)
DJ['<KELTNER_H>'] = keltner.keltner_channel_hband()
DJ['<KELTNER_L>'] = keltner.keltner_channel_lband()
DJ['<KELTNER_M>'] = keltner.keltner_channel_mband()

DJ['<KELT_L_IND>'] = keltner.keltner_channel_lband_indicator()
DJ['<KELT_H_IND>'] = keltner.keltner_channel_hband_indicator()



bollinger =BollingerBands(close=DJ['<CLOSE>'],window=50,window_dev=3,fillna=True)
DJ['<BOL_H_IND>']=bollinger.bollinger_hband_indicator()
DJ['<BOL_L_IND>']=bollinger.bollinger_lband_indicator()

# macd = MACD(DJ['<CLOSE>'],fillna=True)


DJ['<ATR_24>'] = average_true_range(close=DJ['<CLOSE>'],high=DJ['<HIGH>'],low=DJ['<LOW>'],window=30).apply(lambda e : e/100)
DJ['<ATR_24_MULT>'] =abs(DJ['<SIZE>']) / DJ['<ATR_24>'] 


DJ["<ISGREEN>"] = DJ["<ISGREEN>"].astype(int)



In [23]:
timeFrame = (int(DJ.iloc[1][1][1])-int(DJ.iloc[0][1][1]) ) * 60 + (int(DJ.iloc[1][1][3:5])-int(DJ.iloc[0][1][3:5]))
CandlesInDay = 24 * (60//timeFrame)
timeFrame
def candleToTime(j):
    minuteMult = CandlesInDay//24
    k=j//minuteMult
    sth=timeFrame*(j%minuteMult)
    return '{:02d}:{:02d}:00'.format(k,sth)

uniqueDays = DJ.drop_duplicates(subset='<DATE>')
uniqueDays = pd.DataFrame(uniqueDays)

uniqueDaysCount=uniqueDays.shape[0]

newnumparr = np.full((uniqueDaysCount*CandlesInDay,2),'',dtype=np.object_)


for i in range(uniqueDaysCount):
    for j in range(CandlesInDay):
        newnumparr[(i*CandlesInDay)+j]=[uniqueDays.iloc[i][0],candleToTime(j)]

newDF = pd.DataFrame(newnumparr,columns=['<DATE>','<TIME>'])

newestDF = newDF.merge(DJ,on=['<DATE>','<TIME>'],how='left')


In [24]:
newestDF.drop(range(CandlesInDay),inplace=True)
newestDF.fillna(0.0,inplace=True)

In [25]:
newestDF.describe()

,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<SPREAD>,<ISGREEN>,<SIZE>,<VOLATILITY>,<EMA30>,...,<RSI>,<KELTNER_H>,<KELTNER_L>,<KELTNER_M>,<KELT_L_IND>,<KELT_H_IND>,<BOL_H_IND>,<BOL_L_IND>,<ATR_24>,<ATR_24_MULT>
count,36120.000000,36120.000000,36120.000000,36120.000000,36120.000000,36120.000000,36120.000000,36120.000000,36120.000000,36120.000000,...,36120.000000,36120.000000,36120.000000,36120.000000,36120.000000,36120.000000,36120.000000,36120.000000,36120.000000,3.612000e+04
mean,5434.366548,5442.025133,5426.499352,5434.406459,2155.695792,9.354623,0.387846,0.000415,5441.253644,5430.729255,...,0.393162,5449.448188,5418.405211,5433.926700,0.241667,0.304679,0.004568,0.006755,0.159228,inf
std,2984.705515,2988.431832,2980.868389,2984.718278,2576.079910,18.641429,0.487266,0.211993,2988.018345,2986.861435,...,0.236571,2991.825541,2976.664279,2984.227960,0.428099,0.460278,0.067434,0.081913,0.128594,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.963926,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,5793.775000,5808.675000,5778.300000,5794.100000,143.000000,0.000000,0.000000,-0.055761,5807.675518,5792.193296,...,0.274279,5824.347167,5768.463000,5795.573000,0.000000,0.000000,0.000000,0.000000,0.112784,2.443579e-02
50%,7021.800000,7031.100000,7011.050000,7021.700000,1266.000000,4.000000,0.000000,0.000000,7030.102547,7024.749259,...,0.469437,7044.560000,7003.930000,7025.742667,0.000000,0.000000,0.000000,0.000000,0.162774,3.348140e-01
75%,7346.225000,7354.400000,7337.800000,7346.100000,3191.000000,9.000000,1.000000,0.063498,7353.400681,7346.161466,...,0.565531,7362.504833,7328.775667,7345.881000,0.000000,1.000000,0.000000,0.000000,0.208091,7.909651e-01
max,7899.400000,7909.400000,7889.900000,7899.400000,22862.000000,112.000000,1.000000,3.536502,7908.401052,7858.856449,...,0.875402,7856.245333,7823.671333,7839.821333,1.000000,1.000000,1.000000,1.000000,1.248644,inf


In [36]:
newestDF.head(10)

,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<SPREAD>,<ISGREEN>,<SIZE>,...,<RSI>,<KELTNER_H>,<KELTNER_L>,<KELTNER_M>,<KELT_L_IND>,<KELT_H_IND>,<BOL_H_IND>,<BOL_L_IND>,<ATR_24>,<ATR_24_MULT>
24,2016.07.08,00:00:00,6537.8,6537.8,6528.9,6528.9,6.0,110.0,0.0,-0.136317,...,0.438245,6558.906061,6521.887879,6540.396970,0.0,0.0,0.0,0.0,0.0,inf
25,2016.07.08,01:00:00,6538.4,6538.9,6523.9,6527.4,249.0,107.0,0.0,-0.168520,...,0.430342,6557.752778,6521.319444,6539.536111,0.0,0.0,0.0,0.0,0.0,inf
26,2016.07.08,02:00:00,6527.9,6530.4,6526.6,6526.9,242.0,110.0,0.0,-0.015321,...,0.427588,6555.753846,6521.538462,6538.646154,0.0,0.0,0.0,0.0,0.0,inf
27,2016.07.08,03:00:00,6527.1,6549.6,6522.5,6544.6,1327.0,8.0,1.0,0.267396,...,0.539393,6556.485714,6520.842857,6538.664286,0.0,0.0,0.0,0.0,0.0,inf
28,2016.07.08,04:00:00,6544.6,6546.6,6538.1,6545.1,1778.0,8.0,1.0,0.007639,...,0.542100,6556.171111,6521.771111,6538.971111,0.0,0.0,0.0,0.0,0.0,inf
29,2016.07.08,05:00:00,6545.1,6545.1,6528.1,6534.6,1429.0,7.0,0.0,-0.160683,...,0.478789,6555.968750,6521.593750,6538.781250,0.0,0.0,0.0,0.0,0.0,inf
30,2016.07.08,06:00:00,6534.6,6537.1,6519.6,6522.1,1394.0,8.0,0.0,-0.191656,...,0.416713,6555.250980,6520.839216,6538.045098,0.0,0.0,0.0,0.0,0.0,inf
31,2016.07.08,07:00:00,6522.1,6526.6,6519.6,6524.6,1137.0,8.0,1.0,0.038317,...,0.432480,6553.881481,6520.603704,6537.242593,0.0,0.0,0.0,0.0,0.0,inf
32,2016.07.08,08:00:00,6524.6,6531.1,6524.1,6529.6,1441.0,8.0,1.0,0.076574,...,0.463554,6552.901754,6520.638596,6536.770175,0.0,0.0,0.0,0.0,0.0,inf
33,2016.07.08,09:00:00,6529.6,6537.6,6524.1,6531.6,2106.0,7.0,1.0,0.030620,...,0.475853,6552.486667,6520.486667,6536.486667,0.0,0.0,0.0,0.0,0.0,inf


In [26]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [27]:

import multiprocessing
import mp


df = newestDF.drop(['<DATE>','<TIME>'],axis=1)
threadCount = 2

trainNps=[]
labelValues = []

for i in range(threadCount): trainNps.append([])
for i in range(threadCount): labelValues.append([])

maxValues = df.max()

lenDFthread =  len(df) / threadCount
threads = []
p = multiprocessing.Pool(threadCount)
out=p.starmap(mp.thread_function,[(df.iloc[int(index*lenDFthread):int((1+index)*lenDFthread-1)],index,CandlesInDay) for index in range(threadCount)])



In [28]:
np.array(out,dtype=np.object_).shape

(2, 2, 8272)

In [29]:
datas=[]
labs=[]
for i in range(threadCount): 
    datas = datas+out[i][0]
    labs = labs + out[i][1]


In [30]:
len(datas),len(labs)

(16544, 16544)

In [31]:

l = len(datas)
windows2= np.array(datas)

labs = np.array(labs)


train_data = windows2[:int(l*.75)]
val_data = windows2[int(l*.75)+1:]



train_labels = labs[:int(l*.75)].astype(np.uint8)
val_labels = labs[int(l*.75)+1:].astype(np.uint8)


len(train_data),len(train_labels)


(12408, 12408)

In [32]:
len(val_data),len(val_labels)

(4135, 4135)

In [33]:
pd.DataFrame(labs,columns=['signal']).value_counts()/len(labs)

signal
2         0.557785
1         0.222679
0         0.219536
dtype: float64

In [34]:
tf.random.set_seed(42)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D,LSTM, ConvLSTM1D,Flatten,SimpleRNN,GRU,AveragePooling2D

tf.random.set_seed(42)

model_rnn = keras.Sequential([
    LSTM(32,input_shape=train_data[0].shape,dropout=.1,return_sequences=1),
    LSTM(64,dropout=.1,return_sequences=1),
    LSTM(128,dropout=.1,return_sequences=0),
    # Flatten(),
    Dense(100,activation='relu'),
    Dense(3,activation='sigmoid'),

])

model_rnn.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = 'adam',
    metrics=['accuracy']
)

model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 3, 32)             4992      
                                                                 
 lstm_1 (LSTM)               (None, 3, 64)             24832     
                                                                 
 lstm_2 (LSTM)               (None, 128)               98816     
                                                                 
 dense (Dense)               (None, 100)               12900     
                                                                 
 dense_1 (Dense)             (None, 3)                 303       
                                                                 
Total params: 141,843
Trainable params: 141,843
Non-trainable params: 0
_________________________________________________________________


In [35]:
model_rnn.fit(train_data,train_labels,validation_data=(val_data,val_labels),batch_size=64,epochs=200,verbose=2)

Epoch 1/200
194/194 - 11s - loss: 1.0727 - accuracy: 0.5136 - val_loss: 0.9998 - val_accuracy: 0.6832 - 11s/epoch - 59ms/step
Epoch 2/200
194/194 - 5s - loss: 1.0423 - accuracy: 0.5160 - val_loss: 0.9474 - val_accuracy: 0.6832 - 5s/epoch - 28ms/step
Epoch 3/200
194/194 - 6s - loss: 1.0318 - accuracy: 0.5160 - val_loss: 0.9216 - val_accuracy: 0.6832 - 6s/epoch - 33ms/step
Epoch 4/200
194/194 - 4s - loss: 1.0289 - accuracy: 0.5160 - val_loss: 0.9105 - val_accuracy: 0.6832 - 4s/epoch - 20ms/step
Epoch 5/200
194/194 - 4s - loss: 1.0283 - accuracy: 0.5160 - val_loss: 0.9049 - val_accuracy: 0.6832 - 4s/epoch - 20ms/step
Epoch 6/200
194/194 - 4s - loss: 1.0282 - accuracy: 0.5160 - val_loss: 0.9020 - val_accuracy: 0.6832 - 4s/epoch - 20ms/step
Epoch 7/200
194/194 - 4s - loss: 1.0282 - accuracy: 0.5160 - val_loss: 0.9022 - val_accuracy: 0.6832 - 4s/epoch - 20ms/step
Epoch 8/200
194/194 - 4s - loss: 1.0282 - accuracy: 0.5160 - val_loss: 0.9016 - val_accuracy: 0.6832 - 4s/epoch - 20ms/step
Epoch 

KeyboardInterrupt: 